# Libraries

In [111]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, date
from collections import defaultdict
import json

# finance
import yfinance as yf
""" 
# uncomment for first time installation
!git clone https://github.com/joqueka/bf4py.git
import sys
sys.path.append("bf4py")
"""
import bf4py

# efficiency
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Data Import

### get list of all German listed companies

https://topforeignstocks.com/stock-lists/the-list-of-listed-companies-in-germany/

In [112]:
traded_companies_path = '../data/Complete-List-of-Listed-Companies-on-Germany-Frankfurt-Stock-Exchange-Jan-2021.xlsx'

traded_companies = pd.DataFrame()

# List the sheet names you want to read
sheet_names_to_read = [1, 2, 3]  # 0-based index, or you can use sheet names as strings

# Loop through each sheet to read and combine data
for sheet_name in sheet_names_to_read:
    # Read the sheet into a DataFrame, skipping the first 7 rows
    df = pd.read_excel(traded_companies_path, sheet_name=sheet_name, skiprows=range(0, 7), usecols=range(0,6))
    
    # Concatenate the newly read DataFrame to the combined DataFrame
    traded_companies = pd.concat([traded_companies, df], ignore_index=True, axis=0)

# only keep german companies
traded_companies = traded_companies[traded_companies['Country'] == 'Germany']

traded_companies.head()


,ISIN,Trading Symbol,Company,Sector,Subsector,Country
0,DE0005545503,DRI,1+1 DRILLISCH AG O.N.,Telecommunication,Wireless Communication,Germany
1,DE0005118806,TGT,11 88 0 SOLUTIONS AG,Telecommunication,Telecommunication Services,Germany
2,DE0005167902,UUU,3U HOLDING AG,Industrial,Industrial Products & Services,Germany
3,DE000A2YN801,4BSB,4BASEBIO AG NA O.N.,Pharma & Healthcare,Biotechnology,Germany
4,DE000A14KL72,VSC,4SC AG,Pharma & Healthcare,Biotechnology,Germany


### get historical data

In [114]:
# Initialize a defaultdict to store the results
final_dict = defaultdict(dict)

# Create an instance of the General class
general_instance = bf4py.general.General()

# Initialize a list to store ISINs that are not found
not_found_isins = []

# Loop through the list of ISINs
for ISIN in tqdm(traded_companies_list, desc='Getting Share Prices'):
    try:
        history = general_instance.eod_data(
                        isin=ISIN,
                        min_date=datetime.strptime('2010-01-01', '%Y-%m-%d').date(),
                        max_date=date.today()
                        )
        
        # Populate the dictionary
        for record in history:
            final_dict[record['date']][ISIN] = record['close']
    except Exception as e:
        not_found_isins.append(ISIN)

# Convert the dictionary to a DataFrame
final_df = pd.DataFrame.from_dict(final_dict, orient='index')
final_df.index = pd.to_datetime(final_df.index)
final_df.sort_index(inplace=True)

Getting Share Prices ...: 100%|██████████| 449/449 [01:11<00:00,  6.25it/s]


In [137]:
from tqdm import tqdm
import json
import pandas as pd
import numpy as np

# Initialize an empty DataFrame to store the quarterly market cap
quarterly_market_cap_df = pd.DataFrame()

# Create an instance of the Equities class
equity_instance = bf4py.equities.Equities()

# Initialize a list to store problematic ISINs
problematic_isins = []

# Loop through each column (ISIN) in the final_df
for ISIN in tqdm(final_df.columns, desc='Calculating quarterly Market Cap'):
    try:
        # Validate ISIN format (basic validation)
        if len(ISIN) != 12:
            print(f"Skipping invalid ISIN format: {ISIN}")
            continue

        # Resample to get the average share price per quarter
        avg_price_per_quarter = final_df[ISIN].resample('Q').mean()
        
        # Check for NaN
        if pd.isna(avg_price_per_quarter).all():
            avg_price_per_quarter = 1

        # Get the number of shares using the key_data method
        key_data = equity_instance.key_data(isin=ISIN)
        num_shares = key_data.get('numberOfShares', 1)  # Default to 1 if not found
        
        # Calculate the quarterly market cap
        quarterly_market_cap = avg_price_per_quarter * num_shares

        # Check for problematic calculations
        if quarterly_market_cap.equals(num_shares):
            quarterly_market_cap = 1
        
        # Add to the quarterly_market_cap_df DataFrame
        quarterly_market_cap_df[ISIN] = quarterly_market_cap
    except json.JSONDecodeError:
        print(f"JSONDecodeError for ISIN: {ISIN}")
        problematic_isins.append(ISIN)
    except Exception as e:
        print(f"An unknown error occurred for ISIN: {ISIN}. Error: {e}")
        problematic_isins.append(ISIN)

# Output problematic ISINs
print("Problematic ISINs:", problematic_isins)


Calculating quarterly Market Cap:  12%|█▏        | 41/331 [00:06<00:30,  9.38it/s]

JSONDecodeError for ISIN: DE0005419105
JSONDecodeError for ISIN: DE000A2G9M17
JSONDecodeError for ISIN: DE0005313704
JSONDecodeError for ISIN: DE0007257503


Calculating quarterly Market Cap:  14%|█▎        | 45/331 [00:06<00:22, 12.97it/s]

JSONDecodeError for ISIN: DE0005407100
JSONDecodeError for ISIN: DE0005403901
JSONDecodeError for ISIN: DE000CBK1001
JSONDecodeError for ISIN: DE000A288904


Calculating quarterly Market Cap:  14%|█▍        | 47/331 [00:06<00:20, 14.19it/s]

...

JSONDecodeError for ISIN: DE000A0V9LA7
JSONDecodeError for ISIN: DE000A0MZ4B0
JSONDecodeError for ISIN: DE0005533400
JSONDecodeError for ISIN: DE000A0XYG76


Calculating quarterly Market Cap:  91%|█████████ | 300/331 [00:28<00:01, 17.58it/s]

JSONDecodeError for ISIN: DE0005649503
JSONDecodeError for ISIN: DE0005494165
JSONDecodeError for ISIN: DE000A161077
JSONDecodeError for ISIN: DE000A0LR9G9


Calculating quarterly Market Cap:  96%|█████████▌| 318/331 [00:32<00:02,  5.06it/s]/tmp/ipykernel_1031/2795882821.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  quarterly_market_cap_df[ISIN] = quarterly_market_cap
Calculating quarterly Market Cap:  96%|█████████▋| 319/331 [00:34<00:08,  1.46it/s]/tmp/ipykernel_1031/2795882821.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  quarterly_market_cap_df[ISIN] = quarterly_market_cap
Calculating quarterly Market Cap:  97%|█████████▋| 320/331 [00:34<00:05,  1.92it/s]/tmp/ipyker

Problematic ISINs: ['DE0005419105', 'DE000A2G9M17', 'DE0005313704', 'DE0007257503', 'DE0005407100', 'DE0005403901', 'DE000CBK1001', 'DE000A288904', 'DE0005439004', 'DE0006062144', 'DE000A2LQUA5', 'DE000A0LAUP1', 'DE0005470306', 'DE0007100000', 'DE0005498901', 'DE000A2E4K43', 'DE0005146807', 'DE000A0XFSF0', 'DE000A2GS5D8', 'DE0005810055', 'DE0007480204', 'DE0005552004', 'DE000A0HN5C6', 'DE0006305006', 'DE000A2NBVD5', 'DE0005157101', 'DE0005550602', 'DE0005550636', 'DE000A1TNUT7', 'DE000A14KRD3', 'DE0008019001', 'DE0005557508', 'DE0005565204', 'DE000DWS1007', 'DE000ENAG999', 'DE0005659700', 'DE0005854343', 'DE0005654933', 'DE0005677108', 'DE0007856023', 'DE000A11Q059', 'DE0006095003', 'DE000A1X3WX6', 'DE000EVNK013', 'DE0005664809', 'DE0005772206', 'DE0007201907', 'DE000FTG1111', 'DE0005774103', 'DE0005773303', 'DE000A0Z2ZZ5', 'DE0005785802', 'DE0005785604', 'DE000A0JJTG7', 'DE0006602006', 'DE0005495626', 'DE000A0LD6E6', 'DE0005800601', 'DE000A161N30', 'DE000A2E4T77', 'DE000A0S8488', 'DE0

In [ ]:
# Create a mapping dictionary from the company_info_df
isin_to_symbol = dict(zip(traded_companies['ISIN'], traded_companies['Trading Symbol']))

# Rename the columns in the final_df
final_df.rename(columns=isin_to_symbol, inplace=True)
quarterly_market_cap_df.rename(columns=isin_to_symbol, inplace=True)

# save file
final_df.to_csv('../data/closing_prices.csv')
quarterly_market_cap_df.to_csv('../data/market_cap.csv')

In [136]:
print(quarterly_market_cap_df.shape)
quarterly_market_cap_df.head()

(56, 92)


,DE0005545503,DE0005118806,DE0005167902,DE000A1TNNN5,DE000A0KFKB3,DE000A1EWWW0,DE0005103006,DE000A0WMPJ6,DE0005110001,DE0008404005,...,DE000A2AA204,DE0005228779,DE000A0LR9G9,DE000A2QEFA1,DE0001218063,DE000A1EWVY8,DE000A0EQ578,DE000A0XYHT5,DE000A0B9N37,DE000A12UP29
2010-03-31,8.903177e+08,2.575260e+08,2.806188e+07,7.768866e+07,6.049872e+07,6.779257e+09,1.524161e+08,2.652768e+09,1.935204e+07,3.428993e+10,...,7.015330e+07,1.104534e+07,NaN,NaN,9.715313e+07,NaN,2.974667e+07,NaN,1.364793e+08,3.086524e+07
2010-06-30,9.125305e+08,2.287634e+08,2.522507e+07,8.523822e+07,6.719215e+07,7.532757e+09,2.194765e+08,2.712456e+09,2.070961e+07,3.433660e+10,...,7.285048e+07,1.183619e+07,NaN,NaN,8.714772e+07,NaN,3.033780e+07,NaN,1.531712e+08,3.248097e+07
2010-09-30,8.770135e+08,1.834096e+08,2.672730e+07,8.389125e+07,6.487969e+07,7.540623e+09,2.373901e+08,2.440746e+09,2.299912e+07,3.429987e+10,...,6.991479e+07,1.283057e+07,NaN,NaN,8.119491e+07,NaN,3.067373e+07,NaN,1.885545e+08,3.156891e+07
2010-12-31,1.027600e+09,1.796059e+08,2.699677e+07,8.582479e+07,6.659842e+07,8.678025e+09,2.957979e+08,2.743376e+09,2.953895e+07,3.574045e+10,...,6.820223e+07,1.277743e+07,NaN,NaN,9.589121e+07,1.505515e+08,3.309885e+07,NaN,1.707014e+08,3.244135e+07
2011-03-31,1.179296e+09,2.032777e+08,2.493017e+07,8.114853e+07,6.660923e+07,8.354489e+09,3.398158e+08,3.454518e+09,4.135065e+07,4.028928e+10,...,6.763338e+07,1.293416e+07,NaN,NaN,9.446690e+07,1.373525e+08,3.985949e+07,NaN,1.711793e+08,4.387923e+07


In [120]:
final_df.head()

,DE0005545503,DE0005118806,DE0005167902,DE000A1TNNN5,DE000A0KFKB3,DE000A1EWWW0,DE0005103006,DE000A0WMPJ6,DE0005110001,DE0008404005,...,DE000A1MMEV4,DE000A161NR7,DE000A0KEXC7,DE0005895403,DE000A2G8X31,DE000A0BVU28,DE0007846867,DE000A2E4LE9,DE000A255G36,DE0005156004
2010-01-04,4.96,9.122715,0.780,25.30,1.846531,38.505,2.525,23.840,3.45,88.54,...,NaN,NaN,11.35,5.649299,NaN,3.32,3.59,NaN,NaN,1.253154
2010-01-05,4.92,8.818263,0.830,25.00,1.856082,39.720,2.670,23.925,3.45,88.81,...,NaN,NaN,11.44,5.615061,NaN,3.32,3.60,NaN,NaN,1.305368
2010-01-06,4.78,8.985761,0.840,24.83,1.804082,39.400,2.810,24.900,3.90,89.50,...,NaN,NaN,11.49,5.759839,NaN,3.39,3.64,NaN,NaN,1.348881
2010-01-07,4.64,8.887233,0.820,25.00,1.847592,39.745,2.650,25.090,3.86,88.47,...,NaN,NaN,11.35,5.649299,NaN,3.40,3.64,NaN,NaN,1.314071
2010-01-08,4.57,8.966055,0.791,25.20,1.919755,39.600,2.720,24.915,3.76,87.99,...,NaN,NaN,11.50,5.820490,NaN,3.45,3.45,NaN,NaN,1.305368


# Test Area

In [133]:
test = np.array([np.NaN, np.NaN, np.NaN])
np.nanmean(test)

/tmp/ipykernel_1031/1163965101.py:2: RuntimeWarning: Mean of empty slice
  np.nanmean(test)


nan